# <span style="color:teal;">CIS 211 Project 8:  &nbsp; Bears and Fish</span>

##### Due 11:00 P.M. Thursday Mar 16

##### <span style="color:red">Group Members:</span>

Group work is allowed for this project, but groups are exepcted to do more work (see markdown cells at the end of the notebook for suggestions and ideas).  

If you work in a group only one group member should upload this notebook to Canvas.  Edit this cell to include the name and DuckID for each member of the group:

**Name:**  Andrew Letz 951526623

**Name:**

**Name:**

##  <span style="color:teal;">Overview</span> 

The last project this term is a cellular automaton style biological simulation.  

The sytem is described in Chapter 11 of the Miller and Ranum text.  **Read the description of the simulation in the textbook before you start working on this project.**  Your code will be quite a bit different than the code in the book, but the rules of the simulation are the same, and you need to know how objects interact.

The project has three main parts:
* A class named World that defines a 2D grid where cells are accessed according to their row and column coordinates
* Classes for Bear and Fish, the organisms that can inhabit the world
* Two top level functions, `wbf` and `step_system`; the first will make a world and populate it at random with bears and fish, and the second will run a simulation using that world.

We're written `step_system` for you -- it's in the last code cell in this notebook -- but you need to design, implement, and test the classes and the `wbf` function.

##  <span style="color:teal;">Libraries</span> 

Your program will need to use `numpy` (a numeric processing library) and `random` (the builtin random number generator module).  You can import additional modules if you wish; add the import statements to this code cell.

In [1]:
import numpy as np
import random

##  <span style="color:teal;">Event Log</span> 

A useful debugging technique is to save descriptions of events in a log.  Any code in the simulator, whether it is the top level function or a method in one of your classes, can call a function named `log`, passing it a description of an event that just occurred.  Examples might be 
```
new Fish in cell (x,y)
```
or 
```
Bear in cell (x,y) eats Fish in cell (x,y)
```

The function below uses a global variable named `logging`.  If you want to take advantage of the `log` function:
* edit the cell to set `logging` to True
* add calls to `log` at various points in your methods

Later, when doing large scale simulations, you can turn off logging simply by setting `logging` to False before you call the top level simulation function.

**Note:** &nbsp; you can change the definition of the `log` function however you want, _e.g._ you can have it write log messages to a file or printadditional information.

In [2]:
logging = False

def log(message):
    if logging:
        print(message)

##  <span style="color:teal;">World (20 points)</span>

Describe your World class in the following markdown cell, and write the code for the class in the code cell below the markdown cell.

The minimum requiremens for this class are:
* the constructor should be passed the grid size (number of rows and columns)
* all cells should contain `None` when the world is initialized; later they can contain references to Fish or Bear objects
* include a method named `biota` that returns a list of Fish and Bear objects currently in the grid

You should also define the methods that allow you to access a cell or store a value in a cell using Python's indexing operator.  For example, if `w` is a World object, `w[i,j]` should return the object in row `i`, column `j` (which could be None).  You should also be able to store an item in the grid by using `w[i,j]` on the left side of an assignment statement.

You can have additional instance methods, class variables, or class methods.  Make sure you describe any new additions in the documentation.

**Note:** &nbsp; There are no auto-grader tests for the World class; tests for this class will be done as part of the tests for the Bear and Fish classes.

##### <span style=color:red>Documentation</span> 

**Important** Write your documentation in the following markdown cell.  Do not delete or move this cell.

The World class implements a two dimensional numpy array along with a dictionary in order to track a biological simulation of whatever objects/animals desired.
- Index through world[row][col]
- Return a list with all objects from the class-built dictionary of objects with worldname.biota()
- Find width (columns) and height (rows) with worldname.width() and worldname.height()

Setting an item automatically adds it to the built-in dictionary and if an item is set to None it is removed.

##### <span style=color:red>Code</span> 

**Important** Write the definition of your World class in the following code cell. Do not delete or move this cell.

In [3]:
class World():
    def __init__(self, width, height):
        self._width = width
        self._height = height
        
        self._a = np.array( [None] * (height*width)).reshape(height,width)
        self._bdict = {}
    
    def width(self):
        return self._width

    def height(self):
        return self._height
    
    def __repr__(self):
        return repr(self._a)
    
    def __getitem__(self, loc):
        return self._a[loc]
    
    def __setitem__(self, loc, val):
        self._a[loc] = val
        if val != None:
            self._bdict[loc] = val
        else:
            #print('--- DELETING', loc)
            del self._bdict[loc]
        
    def biota(self):
        return [x for x in self._bdict.values()]

## <span style="color:teal;">Fish and Bears</span>

To run a simulation we need to add a random collection of animals to the world.  The two types of animals in this simulation are fish and bears, and you will write class definitions named Fish and Bear that implement the behaviors of the animals.

### Fish Class

During the simulation a Fish object needs to behave as follows:

(1) Fish are susceptible to overcrowding:  if there are fish in 2 or more neighboring cells the fish dies (it's removed from the simulation)

(2) A fish can reproduce if it has been alive for a certain number of time steps: a random neighboring cell is chosen, and if that cell is empty, a new fish is placed in that cell

(3) A fish can move to another cell:  it picks a random direction, and if the neighboring cell in that direction is unoccupied the fish moves there

The constructor for the Fish class will be passed a reference to a World object and a location, in the form of a tuple with a row number and column number (the object needs to know its location so it can look for other objects in neighboring cells).

The class should include the following methods:
* `live` implements rules 1 and 2 shown above
* `move` implements rule 3
* `location` returns the current grid location (row and column) of the object

Define a class variable named `breed_interval` to specify how many time steps a fish must be alive before it reproduces; the initial value for this variable is 12.


### Bear Class

During the simulation a Bear object needs to behave as follows:

(1) A bear looks for fish in each adjacent cell; if it finds one or more fish it eats one at random 

(2) If a bear has not eaten for certain number of time steps it dies (it's removed from the simulation)

(3) A bear can reproduce if it has been alive for a certain number of time steps: a random neighboring cell is chosen, and if that cell is empty, a new bear is placed in that cell

(4) A bear can move to another cell:  it picks a random direction, and if the neighboring cell in that direction is unoccupied the bear moves there

Define a class variable named `breed_interval` to specify how many time steps a bear must be alive before it reproduces; the initial value for this variable is 8.  Define another class variable named `survive_without_food` to be the number of time steps a bear can live before it dies from starvation; the initial value for this variable is 10.

The constructor will be passed a reference to a World object and a location, in the form of a tuple with a row number and column number (the object needs to know its location so it can look for other objects in neighboring cells).

The class should include the following methods:
* `live` implements rules 1, 2, and 3 shown above
* `move` implements rule 4
* `location` returns the current grid location (row and column) of the object

### Animal (Base Class)?

From the descriptions above it should be apparent that fish and bears have some things in common.

For **full credit** on the coding and documnetation portions of this project you should define a class named Animal and use it as the base class for your Fish and Bear classes.  Some things to think about as you design your classes:
* are there behaviors or operations that are common to both and that can be implemented just once in Animal?
* perhaps a behavior or operation can be defined with a default in the Animal class, and then overridden in the derived class?

One strategy you might consider is to write a complete implementation for one class, either Bear or Fish.  After you have debugged the class and it passes its unit tests you'll have a better idea of what to implement in the other class.  Then you can start moving common behaviors to the Animal class while you are writing the other derived class.

You can still receive **partial credit** if you skip the Animal class and simply write completely separate Fish and Bear classes.  None of the unit tests assume there is a class named Animal.

### Details and Hints

**Fish:** &nbsp; The way the simulation is defined a fish might be eaten before the top level simulation calls the `live` method.  Your `live` method should check to make sure the fish is still alive.  The easiest way to do this is to include an instance variable named `_alive` that is set to True when the fish is initialized and set to False when it dies.

You can have additional instance methods, class variables, or class methods.  Make sure you describe any new additions in the documentation.

**Animal:** &nbsp; Keep this in mind when you design the class hierarchy: every object has an attribute named ``__class__`` (with two underscores before and after the name).  It is a reference to the class an object was defined with.  As an example of how to use it, consider what would happen if we want to define `reproduce` in the Animal class so it is inherited by both Fish and Bear classes.  We need to know the value of `breed_interval` in each object's own class.  We can find this value using the expression
```
self.__class__.breed_interval
```
This will be a reference to `Fish.breed_interval` or `Bear.breed_interval`, depending on whether a `reproduce` was called with Fish object or Bear object.


### <span style="color:teal;">Animal (20 points)</span>

##### <span style=color:red>Documentation</span> 

If you implement the Animal class write your documentation in the following markdown cell.  

**Important** Do not delete or move this cell.

--------- Neighbors

Neighbors returns a dictionary of each neighboring cell coordinate for a given location in a given world. This implemets the idea of wrap around, so if a cell is on the top right corner of the world, the value of 'nw' in the dictionary will return the bottom left corner's coordinate.

--------- Animal

The animal class employs a general interface for an animal in a biological simulation.
Instance variables:
    ._world - the world which contains this animal
    ._loc - the tuple location inside this 2d world
    ._age - the amount of steps the animal has been alive
    ._alive - whether or not the animal is alive
    ._neighbors - a dictionary of all neighboring cell coordinates for the animal
    
The methods reproduce(), location(), updateneighbors(), updateloc(), kill() and move() are general enough to be written at this general of a level. live() however is left to subclasses to define as its behavior varies greatly.

##### <span style=color:red>Code</span> 

If you implement the Animal class write the definition in the following code cell

**Important:** &nbsp; Do not delete or move this cell.

In [4]:
def neighbors(loc, world):
    rows, cols = world.height(), world.width()
    directions = dict([('w' , (loc[0], (loc[1] - 1) % cols)), 
                      ('n' , ((loc[0] - 1) % rows, loc[1])),
                      ('e' , (loc[0], (loc[1] + 1) % cols)),
                      ('s' , ((loc[0] + 1) % rows, loc[1])),
                      ('nw' , ((loc[0] - 1) % rows, (loc[1] - 1) % cols)),
                      ('sw' , ((loc[0] + 1) % rows, (loc[1] - 1) % cols)),
                      ('ne' , ((loc[0] - 1) % rows, (loc[1] + 1) % cols)),
                      ('se' , ((loc[0] + 1) % rows, (loc[1] + 1) % cols))])

    return directions

In [5]:
class Animal():
    
    def __init__(self, world, location):
        self._world = world
        self._loc = location
        self._age = 0
        self._alive = True
        
        self._world[location] = self
        self._neighbors = neighbors(self._loc, self._world)

    def reproduce(self, bi):
        self.updateneighbors()
        if self._age > 0:
            if self._age % bi == 0:
                for nloc in [x for x in self._neighbors.values()]:
                    obj = self._world[nloc[0]][nloc[1]]
                    if obj == None:
                        self.__class__(self._world, (nloc[0], nloc[1]))
                        self.updateneighbors()
                        break
    
    def __repr__(self):
        return 'ANML'
        
    def location(self):
        return self._loc
    
    def updateneighbors(self):
        self._neighbors = None
        self._neighbors = neighbors(self._loc, self._world)
    
    def updateloc(self, location):
        self._world[self._loc] = None
        self._world[location] = self
        self._loc = location
        
    def move(self):
        self.updateneighbors()
        if self._alive:
            nlist = [x for x in self._neighbors.values()]
            for nloc in random.sample(nlist, len(nlist)):
                obj = self._world[nloc[0]][nloc[1]]
                if obj == None:
                    #print('-- Moving', self, 'from', self._loc, 'to', nloc)
                    loctuple = (nloc[0], nloc[1])
                    self.updateloc(loctuple)
                    self.updateneighbors()
                    break
    
    def kill(self):
        self._alive = False
        self._world[self._loc] = None
    
    def live(self):
        #overriden by sub-classes
        pass

### <span style="color:teal;">Fish (20 points)</span>

##### <span style=color:red>Documentation</span> 

**Important:** &nbsp; Write the documentation for your Fish class in the following markdown cell.  Do not delete or move this cell.

A subclass of Animal, fish overrides the __repr__ method and the live() method. It also employs a new class variable "breed_interval", which is passed into reproduce() in the parent class Animal.

live() uses the logic laid out in the previous markdown cells.

##### <span style=color:red>Code</span> 

**Important:** &nbsp; Write the Python code for your Fish class in the following code cell.  Do not delete or move this cell.

In [6]:
class Fish(Animal):
    
    breed_interval = 12
    
    def __init__(self, world, location):
        Animal.__init__(self, world=world, location=location)
    
    def __repr__(self):
        return '🐟--'
    
    def live(self):
        self.updateneighbors()
        #print(self._alive, 'before living FISH')
        if self._alive:
            self._age += 1
            ocount = 0
            for nloc in [x for x in self._neighbors.values()]:
                obj = self._world[nloc[0]][nloc[1]]
                if isinstance(obj, Fish):
                    ocount += 1
                    if ocount > 1:
                        #print('- OVERCROWD', self._loc)
                        self.kill()
                        break

        #print(self._alive, 'before reproduction FISH')
        if self._alive:
            self.reproduce(self.breed_interval)

##### <span style="color:red">Autograder Tests:</span>

**Important:** &nbsp;  the code cells in this section will be used by `nbgrader` to run automated tests.  Do not move, delete or alter these cells in any way.

In [7]:
# A new world has no objects
w1 = World(5,5)
assert len(w1.biota()) == 0

# After adding a fish there should be one object
f1 = Fish(w1, (2,2))
assert len(w1.biota()) == 1

# Test the location method
assert f1.location() == (2,2)

In [8]:
# Setting breed_interval to 1 should cause a fish to reproduce when live is called
w2 = World(5,5)
Fish.breed_interval = 1
f2 = Fish(w2, (2,2))
f2.live()
assert len(w2.biota()) == 2

# Reset the interval to original value for remaining tests
Fish.breed_interval = 12

In [9]:
# Make three fish, the one in the middle should die from overcrowding
w3 = World(5,5)
f3 = Fish(w3, (2,2))
Fish(w3, (1,1))
Fish(w3, (3,3))
f3.live()
assert len(w2.biota()) == 2

In [10]:
# When a fish moves it should be within one cell of its original location
w4 = World(5,5)
f4 = Fish(w4, (2,2))
f4.move()
r, c = f4.location()
assert (r,c) != (2,2)
assert abs(r-2) <= 1 and abs(c-2) <= 1

###  <span style="color:teal;">Bear (20 points)</span>

##### <span style=color:red>Documentation</span> 

**Important:** &nbsp; Write the documentation for your Bear class in the following markdown cell.  Do not delete or move this cell.

A subclass of Animal, bear overrides the __repr__ method and the live() method. It also employs two new class variables "breed_interval" and "survive_without_food";  the former which defines the time before a new bear is spawned in a neighboring cell to the current bear, and the latter which defines a time before the bear should die of hunger.

live() uses the logic laid out in the previous markdown cells.

##### <span style=color:red>Code</span> 

**Important:** &nbsp; Write the Python code for your Bear class in the following code cell.  Do not delete or move this cell.

In [11]:
class Bear(Animal):
    
    survive_without_food = 10
    breed_interval = 12
    
    def __init__(self, world, location):
        Animal.__init__(self, world=world, location=location)
        self._hunger = 1
    
    def __repr__(self):
        return '<🐻>'
    
    def live(self):
        self.updateneighbors()
        #print(self._alive, 'before living BEAR')
        if self._alive:
            self._age += 1
            self._hunger += 1

            for nloc in [x for x in self._neighbors.values()]:
                obj = self._world[nloc[0]][nloc[1]]
                if isinstance(obj, Fish):
                    #print('-- EATING', nloc[0], nloc[1])
                    self._world[obj.location()].kill()
                    self._hunger -= 1
                    self.updateneighbors()
                    break

            if self._hunger > self.survive_without_food:
                #print('-- HUNGER', self._loc)
                self.kill()
        
        #print(self._alive, 'before reproduction BEAR')
        if self._alive:
            self.reproduce(self.breed_interval)

##### <span style="color:red">Autograder Tests:</span>

**Important:** &nbsp;  the code cells in this section will be used by `nbgrader` to run automated tests.  Do not move, delete or alter these cells in any way.

In [12]:
# Test the Bear constructor and location method
w1 = World(5,5)
b1 = Bear(w1, (1,1))
assert len(w1.biota()) == 1
assert b1.location() == (1,1)

In [13]:
# Repeat the reproduction test for Bears
w2 = World(5,5)
Bear.breed_interval = 1
b2 = Bear(w2, (2,2))
b2.live()
assert len(w2.biota()) == 2
Bear.breed_interval = 8

In [14]:
# Make fish for the bear to eat, count the number of objects after eating
w3 = World(5,5)
b3 = Bear(w3, (2,2))
Fish(w3, (1,1))
Fish(w3, (3,3))
b3.live()
assert len(w2.biota()) == 2

In [15]:
# Setting the survival limit to 1 should cause a Bear to starve 
w4 = World(5,5)
Bear.survive_without_food = 1
b4 = Bear(w4, (2,2))
b4.live()
assert len(w4.biota()) == 0

In [16]:
# Repeat the move test for bears
w5 = World(5,5)
b5 = Bear(w5, (2,2))
b5.move()
r, c = b5.location()
assert (r,c) != (2,2)
assert abs(r-2) <= 1 and abs(c-2) <= 1

##  <span style="color:teal;">The `wbf` Function (10 points)</span>

Fill in the body of the `wbf` function so it returns a new World object with the specified number of rows and columns and with the specified number of Bear and Fish objects at random locations.

When we grade your project we will call `wbf` to make a World object and then use the main loop (implemented by `step_system`) to run the simulation.

**There is no documentation requirement for the `wbf` function.**

In [17]:
def wbf(nrows, ncols, nbears, nfish):
    '''
    Return a new World object with the specified number of Bear and Fish objects.
    '''
    world = World(ncols, nrows)
    
    b_count = 0
    while b_count < nbears:
        row_b = round((nrows - 1) * random.random())
        col_b = round((ncols - 1) * random.random())
        if world[row_b][col_b] == None:
            b = Bear(world, (row_b, col_b))
            b_count += 1
        
    f_count = 0
    while f_count < nfish:
        row_f = round((nrows - 1) * random.random())
        col_f = round((ncols - 1) * random.random())
        if world[row_f][col_f] == None:
            f = Fish(world, (row_f, col_f))
            f_count += 1
             
    return world

##### <span style="color:red">Autograder Tests:</span>

**Important:** &nbsp;  the code cells in this section will be used by `nbgrader` to run automated tests.  Do not move, delete or alter these cells in any way.

In [18]:
w = wbf(15,15,3,12)

dct = { Bear: 0, Fish: 0 }
for x in w.biota():
    dct[x.__class__] += 1

assert dct[Bear] == 3
assert dct[Fish] == 12

##  <span style="color:teal;">The `step_system` Function</span>

We've written this function for you.  It will run a single time step of the simulation.  Pass it a World object containing a grid populated with Bear and Fish objects and it will (a) see which animals survive, then (b) allow all the animals to move to a new location.

In [19]:
def step_system(world):
    for x in world.biota():
        x.live()
    for x in world.biota():
        x.move()

In [20]:
# Indefinite simulation
Bear.survive_without_food = 10
Bear.breed_interval = 15
Fish.breed_interval = 2
test_world = wbf(11, 11, 1, 40)

In [21]:
# Keep running this to test
step_system(test_world)
test_world

array([[None, None, None, 🐟--, None, None, None, None, None, None, None],
       [None, None, None, None, None, None, 🐟--, None, 🐟--, None, None],
       [🐟--, None, None, None, None, 🐟--, None, None, None, 🐟--, None],
       [None, None, None, None, None, 🐟--, None, None, None, None, None],
       [🐟--, None, None, None, None, None, None, None, None, None, None],
       [None, 🐟--, None, None, 🐟--, None, 🐟--, None, 🐟--, None, 🐟--],
       [None, None, None, None, 🐟--, None, None, None, None, None, 🐟--],
       [None, None, 🐟--, None, None, None, None, 🐟--, None, None, None],
       [None, 🐟--, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, 🐟--, None, None, None, None, None, None],
       [None, None, None, None, <🐻>, 🐟--, None, None, None, None, None]], dtype=object)

##  <span style="color:teal;">Experiments (10 points)</span>

Run some experiments with the top level simulation loop and describe the results in the markdown cell below.  Some ideas of things to try:
* The settings for the Bear class `breed_interval` and `survive_without_food` variables come from the textbook.  Will the world ever run out of bears with these settings?
* Change the settings so `Bear.breed_interval` is larger than `Bear.survive_without_food`.  How does that change the outcome?
* Set the `breed_interval` counter for the Fish class to a smaller number (e.g. 4) so the world has more fish.  What effect does that have?
* Write a function that runs the simulation for a specified number of generations, or until there are no more objects left in the grid.  What combination of parameters leads to the largest number of time steps before the simulation halts?

                                ----------------- INDIVIDUAL TESTS --------------------

   - With the defaults from the textbook (8 ticks for breed interval, and 10 for survive without food) there is a neverending population of bears that continue to reproduce before dying. This population varies from 2-4 bears.

   - If the breed interval is higher than the survive without food timer (as in my default values of 10 and 12), then the bear population will eventually die out regardless of the number of fish. They seem to not be able to reproduce fast enough with default values to escape the bears. This most likely happens because the bears both eat and move in the same time step, causing them to catch up to fish over time and eventually wipe them out.
    
   - Even with fish breeding every 4 ticks they still go extinct from bears leading to an empty simulation. Even as little as 1 to 2 bears to 20 starting fish will still end with bears overwhelming the simulation. This is probably due to the fact that fish can overcrowd and the fact that bears are so much more aggresive.
    
   - Completely removing the overcrowding feature STILL results in bears exterminating the planet.

   - The size of the planet needs to be as high as 12x12, with a 4 breed interval on fish, and only a single bear, to cause fish to be the sole occupants of the simulation. 

                                 ------------ BEST SIMULATION AWARD GOES TO ------------
                                            
Reaching around 3500 time steps before I decided this would most likely continue forever, this simulation seems to hold indefinitely without an end in sight for bears or fish. Settings:

- Overcrowding enabled
- Bear.survive_without_food = 10
- Bear.breed_interval = 15
- Fish.breed_interval = 2
- Starting size 11x11
- Starting bears - only one!
- Starting fish - 40

This simulation not only lasts a large amount of time, but also keeps the bear population low (1-5ish) and the fish population high. The most realistic by far of all tested simulations.

##  <span style="color:teal;">Projects for Groups and/or Extra Credit</span>

Here are some suggestions for ways to extend the simulation.  We will consider other types of extensions -- send a request to `conery@uoregon.edu` with your proposal.

Groups with three people should implement two extensions.  Groups with two people can choose either extension, or do both for extra credit.

* Implement the Plant class described in Section 10.7 of the textbook, and modify the Fish class so fish eat plants and die if they don't find enough food.

* If you implement Plant, how does that affect the class hierarchy?  Is there an even more general class called Organism, with Plants and Animals subtypes of organism?

* Experiment with data structures: make a second version of the World class, but use a list-of-lists approach to making the grid.  Which is more efficient, a list of lists or a numpy array?  Which scales better when larger worlds are used in the simulation?

* Implement a GUI using `tkinter` that is similar to the Solar System GUI.  A canvas should display the world along with images for the Bear and Fish objects (you can download `Bear.gif` and `Fish.gif` from Canvas).  Use a spinbox or text entry box to specify the number of Bear and Fish objects and the number of time steps to run.  Include a Run button to start the simulation.
